In [57]:
import tensorflow as tf
import numpy as np
import glob
import os
import sys
import pandas as pd
import cv2 as cv
from skimage import io
from skimage import transform as tr
from skimage import exposure as ex

In [29]:
from scipy.ndimage import zoom


def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # width and height of the zoomed image
    zh = int(np.round(zoom_factor * h))
    zw = int(np.round(zoom_factor * w))

    # for multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # zooming out
    if zoom_factor < 1:
        # bounding box of the clip region within the output array
        top = (h - zh) // 2
        left = (w - zw) // 2
        # zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # zooming in
    elif zoom_factor > 1:
        # bounding box of the clip region within the input array
        top = (zh - h) // 2
        left = (zw - w) // 2
        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)
        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # if zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [30]:
imagedir = '/home/abake116/Fish/test_stg1/'
modelFullPath = '/home/abake116/Fish/models/inc_retrain_full/output_graph.pb'
labelsFullPath = '/home/abake116/Fish/models/inc_retrain_full/output_labels.txt'

In [31]:
images = glob.glob(imagedir + '*.jpg')

In [32]:
headers = ['image','alb','bet','dol','lag','nof','other','shark','yft']
df = pd.DataFrame(np.nan, index=(range(len(images))),columns=headers)

In [33]:
def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(modelFullPath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [34]:
def run_inference_on_image(image_data):
    
    answer = {'alb':0,'bet':0,'dol':0,'lag':0,'nof':0,'other':0,'shark':0,'yft':0}
    #if not tf.gfile.Exists(imagePath):
    #    tf.logging.fatal('File does not exist %s', imagePath)
    #    return answer
    image_data = tf.gfile.FastGFile(image_data, 'rb').read()

    # Creates graph from saved GraphDef.
    create_graph()

    with tf.Session() as sess:

        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        predictions = sess.run(softmax_tensor,
                               {'DecodeJpeg/contents:0': image_data})
        predictions = np.squeeze(predictions)

        top_k = predictions.argsort()[-8:][::-1]  # Getting top 5 predictions
        f = open(labelsFullPath, 'rb')
        lines = f.readlines()
        labels = [str(w).replace("\n", "") for w in lines]
        for node_id in top_k:
            human_string = labels[node_id]
            score = predictions[node_id]
            answer[human_string] = score
    
        return answer

In [59]:
#Fix this is batch predictions
if __name__ == '__main__':
    
    for i,img in enumerate(images):
        print "image {}".format(i)
        print img
        for a in range(4):
            print 'iteration {}'.format(a)
            image_data = io.imread(img)
            image_data = clipped_zoom(image_data,np.random.uniform(low=1, high=1.5))
            image_data = tr.rotate(image_data,180)
            image_data = ex.adjust_gamma(image_data, np.random.uniform(low=0.5, high=1.5))
            
            #afine_tf = tr.AffineTransform(shear=np.random.uniform(low=-0.3, high=0.3))
            #image_data = tr.warp(image_data, afine_tf)
            io.imsave('augmented.jpg',image_data)
            with tf.Graph().as_default():
                answer = run_inference_on_image('augmented.jpg')
            if a == 0:
                final = answer
            else:
                for itm in final:
                    final[itm] += answer[itm]
        for itm in final:
            final[itm] /= 4
        filename = os.path.basename(img)
        df['image'].iloc[i] = filename
        print final
        for x in answer:
            df[x].iloc[i] = final[x]

image 0
/home/abake116/Fish/test_stg1/img_06332.jpg
iteration 0
iteration 1
iteration 2
iteration 3
{'yft': 0.2761060893535614, 'shark': 0.087264999747276306, 'other': 0.069349810481071472, 'dol': 0.011138921603560448, 'nof': 0.34324482083320618, 'lag': 0.0029978067614138126, 'alb': 0.11516117304563522, 'bet': 0.094736374914646149}
image 1
/home/abake116/Fish/test_stg1/img_05882.jpg
iteration 0


KeyboardInterrupt: 

In [50]:
df.head()

,image,alb,bet,dol,lag,nof,other,shark,yft
0,img_06332.jpg,0.095372,0.077488,0.007811,0.001880,0.359627,0.048093,0.122323,0.287408
1,img_05882.jpg,0.162137,0.018784,0.001621,0.125946,0.020639,0.048369,0.035111,0.587392
2,img_03527.jpg,0.180120,0.110474,0.292287,0.007486,0.102671,0.006103,0.073560,0.227299
3,img_01618.jpg,0.377978,0.085529,0.004163,0.071644,0.261149,0.122199,0.018312,0.059026
4,img_02625.jpg,0.089663,0.030610,0.001672,0.036608,0.175624,0.427981,0.055331,0.182509


In [49]:
df.to_csv('test_aug_4.csv',index=False)